<a href="https://colab.research.google.com/github/edoardochiarotti/class_datascience/blob/main/2023/07_Endogeneity/Practice/07_Endogeneity_exercises.ipynb"
   target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Endogeneity

<img src="https://i.imgflip.com/84g5gd.jpg" width="500">

In [ ]:
# PACKAGES
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
import statistics as st
import pandas as pd
import os
import statsmodels.api as sm
import re
!pip install stargazer
from stargazer.stargazer import Stargazer
!pip install linearmodels
from linearmodels.panel import PanelOLS # conda install -c conda-forge linearmodels

# FUNCTIONS FROM PACKAGES
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression

# SEABORN THEME
scale = 0.4
W = 16*scale
H = 9*scale
sns.set(rc = {'figure.figsize':(W,H)})
sns.set_style("white")

## Outline
- [Exercise 1: Multivariate Linear Regression Model](#Exercise-1:-Multivariate-Linear-Regression-Model)
- [Exercise 2: Interaction Terms](#Exercise-2:-Interaction-Terms)
- [Exercise 3: Fixed Effects](#Exercise-3:-Fixed-Effects)

## Exercise 1: Multivariate Linear Regression Model <a name="Exercise-1:-Multivariate-Linear-Regression-Model"></a>

- Get data on CO2 emissions per capita, income per capita and climate-related tax revenues as done in class:

In [ ]:
# get data
link = "https://www.qogdata.pol.gu.se/data/qog_ei_sept21.xlsx"
df_qog = pd.read_excel(link)

In [ ]:
# get variables
indexes = ["ccodealp","year"]
variabs_co2 = ["edgar_co2gdp","edgar_co2t","edgar_co2pc"]
variabs_control = ["oecd_cctr_gdp"]
variabs = variabs_co2 + variabs_control
df = df_qog.loc[:,np.append(indexes,variabs)]

# make gdp per capita
df["gdp"] = (df["edgar_co2gdp"]/df["edgar_co2t"])**(-1) # billions US dollars
df["pop"] = (df["edgar_co2pc"]/df["edgar_co2t"])**(-1) # millions
df["gdp_pc"] = df["gdp"]/df["pop"] # thousands of US dollars
variabs = np.append(variabs, ["gdp","pop","gdp_pc"])

In [ ]:
# make cross section
df_cross = df.groupby("ccodealp")[variabs].mean().reset_index().dropna()

# put ones into data
df_cross["ones"] = 1

# drop outliers quick and dirty
df_cross = df_cross.loc[df_cross["gdp_pc"] < 80,:]

# maybe logs?
df_cross["ln_gdp_pc"] = np.log(df_cross["gdp_pc"])
df_cross["ln_edgar_co2pc"] = np.log(df_cross["edgar_co2pc"])

In [ ]:
# canned ols
ols_canned_results = sm.OLS.from_formula('ln_edgar_co2pc ~ ln_gdp_pc + oecd_cctr_gdp', df_cross).fit()
ols_canned_results_table = ols_canned_results.summary().tables[1]
ols_canned_results_table

- Use the functions you have written in the exercise Notebook of week 6, called `data_to_matrix` and `OLS_estimator`, to estimate this table by hand:

In [ ]:
# your code here ...

- Our functions should work as we've been good! Note that usually the routines in Stata (and Python I think) for multivariate regressions use the Frisch-Waugh-Lovell Theorem to estimate the single regressors' coefficients. This is much more efficient when the number of regressors become high, or when we have fixed effects. We will not cover it in this class, but you should check the material on [Partitioned Regression](https://static1.squarespace.com/static/558eff8ce4b023b6b855320a/t/573162b320c6478f4961c5bb/1462854326527/ARE_212_Section_4.pdf) by Fiona Burlig, who shows how to do this in R in details and explains why this theorem is so important. We have used it in the class of fixed effects.

## Exercise 2: Interaction Terms <a name="Exercise-2:-Interaction-Terms"></a>

- Use the functions written above (not canned routines) to estimate the log-log model with the interaction term between log of income per capita (`ln_gdp_pc`) and climate change-related tax revenue (`oecd_cctr_gdp`).

In [ ]:
# your code here ...

- Use the functions written above to estimate the level-level model with the interaction term between income per capita (`gdp_pc`) and a dummy variable which equals 1 for countries with high climate change-related tax revenues.

In [ ]:
# your code here


- Comment the results:

- Your answer here: ... 

## Exercise 3: Fixed Effects <a name="Exercise-3:-Fixed-Effects"></a>

- Let's take a subset of the full panel data loaded above, as done in class:

In [ ]:
# drop nas
df_panel = df.dropna()

# from 2010 to 2016
df_panel = df_panel.loc[(df_panel["year"] >= 2010) & (df_panel["year"] <= 2017),:]

# drop outliers quick and dirty
df_panel = df_panel.loc[(df_panel["gdp_pc"] < 65) & (df_panel["edgar_co2pc"] < 20),:]

# lets make it balanced for simplicity
df_panel_agg = df_panel.groupby(['year']).size().reset_index(name='counts')
year_min = df_panel_agg.loc[df_panel_agg.counts == min(df_panel_agg.counts),"year"].values
ccodealp_sub = np.unique(df_panel.loc[df_panel.year == int(year_min),"ccodealp"].values)
df_panel = df_panel.loc[np.isin(df_panel.ccodealp, ccodealp_sub),:]

df_panel_agg = df_panel.groupby(['ccodealp']).size().reset_index(name='counts')
cou_drop = df_panel_agg.loc[df_panel_agg.counts == min(np.unique(df_panel_agg["counts"])),"ccodealp"].values
df_panel = df_panel.loc[~np.isin(df_panel.ccodealp, cou_drop),:]

# maybe logs?
df_panel["ln_gdp_pc"] = np.log(df_panel["gdp_pc"])
df_panel["ln_edgar_co2pc"] = np.log(df_panel["edgar_co2pc"])

# plot
sns.scatterplot(x='ln_gdp_pc', y='ln_edgar_co2pc', data=df_panel, color = "r", s = 20)

- Create a function called `FE_estimator` by updating the function used above (called `OLS_estimator`) with the equations seen in class to estimate the model parameters with the fixed-effect estimator. Some tips:
    - Instead of the usual expression for the OLS estimator, you should have something like this: `beta_hat_FE = (inv(xdata_dem.T @ xdata_dem)) @ (xdata_dem.T @ ydata_dem)`
    - To demean the data, you need to define the matrixes $\iota_T$, $P_{NT}$, etc
    - Note that we you need to add an extra $N$ in the degrees of freedom correction, because all of this is as if we were estimating also $N$ country dummies (even if we don't effectively do it, we can't cheat statistics).

In [ ]:
# your code here ...


- Test this function by estimating the log-log fixed-effect model that regresses the log of CO2 emissions per capita on the log of GDP per capita and the climate change-related tax revenues, using the panel database built above, and comment the coefficient estimates.

In [ ]:
# your code here ...


- Your answer here ...

- Test that the output is the same of the one obtained in class with the canned routine `PanelOLS.from_formula`:

In [ ]:
# your code here ...


- As done in class, compute the demeaned panel database (we called it `demeaned_df`) and use the OLS canned routine `sm.OLS.from_formula` on this database to test if it gives you the same results of your function `FE_estimator` (careful about the intercept). Are the results the same or different? And if they differ, why?

In [ ]:
# your code here ...
